In [36]:
from torch.utils.data import Dataset
import numpy as np
from torch.nn.functional import one_hot
import torch

class M100ImageDataset(Dataset):
    def __init__(self, *, data_path, train=True):
        # Loading data from the 'mnist_compressed.npz' file
        data = np.load(data_path)
        # Reading variables containing the data
        X_test, y_test, X_train, y_train =  data['test_images'], data['test_labels'], data['train_images'], data['train_labels']

        if train:
            self.data = torch.tensor(X_train.astype('float32') / 255., dtype=torch.float32)
            self.labels = one_hot(torch.tensor(y_train), num_classes=100).to(torch.float32)
        else:
            self.data = torch.tensor(X_test.astype('float32') / 255., dtype=torch.float32)
            self.labels = one_hot(torch.tensor(y_test), num_classes=100).to(torch.float32)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [37]:
from torch.utils.data import DataLoader

m100_path = '../data/mnist100/mnist_compressed.npz'

training_data = M100ImageDataset(data_path=m100_path, train=True)
test_data = M100ImageDataset(data_path=m100_path, train=False)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

In [38]:
import torch.nn as nn

class M100Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=(5*12*16), out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=100)
        #self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [39]:
allow_cuda = False


use_cuda = torch.cuda.is_available()
device = torch.device('cuda') if use_cuda and allow_cuda else torch.device('cpu')

loss_fn = torch.nn.CrossEntropyLoss()

model = M100Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [40]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        inputs = inputs.unsqueeze(1)
        #print(inputs.shape)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
print("Finished Training")

KeyboardInterrupt: 